In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
import os
import sqlite3
import warnings

warnings.simplefilter('ignore')

# Objectives of this notebook

The main objectives of this notebook are:
- To get data using web-scraping technique.
- To save the cleaned data as a database using SQLite.

# Data source

- IOC country code: https://raw.githubusercontent.com/johnashu/datacamp/master/medals/Summer%20Olympic%20medalists%201896%20to%202008%20-%20IOC%20COUNTRY%20CODES.csv
- ISO country code: https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv
- Marathon data: https://services.datasport.com/2018/lauf/lamara/

# Method

- Web scraping
- SQL
- Data cleaning
- Data visualization for numerical and geographical data (matplotlib, folium)

# IOC and ISO country code

Obtained results will be saved as csv files and database. In order to store these files, a new folder named "output" is created.

In [6]:
output_path= os.getcwd() + '/output'

if not os.path.exists('output'):
    os.mkdir(output_path)

In the raw marathon data, [IOC country code](https://en.wikipedia.org/wiki/List_of_IOC_country_codes) is used to denote countries. This code will be converted into [ISO country code](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes). In order to do that, lists of country codes need to be obtained. These lists will be obtained as pandas dataframes and saved in a database using SQLite.

In [7]:
# get IOC and ISO country codes 
IOC_code=pd.read_csv('https://raw.githubusercontent.com/johnashu/datacamp/master/medals/Summer%20Olympic%20medalists%201896%20to%202008%20-%20IOC%20COUNTRY%20CODES.csv')
ISO_code=pd.read_csv('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv')

# cleaning data
idx=ISO_code['name'].tolist().index('Iran (Islamic Republic of)')
ISO_code['name'].iloc[idx]='Iran' 

idx=ISO_code['name'].tolist().index('Viet Nam')
ISO_code['name'].iloc[idx]='Vietnam' 

idx=IOC_code['Country'].tolist().index('Burma')
IOC_code['Country'].iloc[idx]='Myanmar' 

idx=IOC_code['Country'].tolist().index('Puerto Rico*')
IOC_code['Country'].iloc[idx]='Puerto Rico' 

In [8]:
ISO_code.head(10)

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN
5,Andorra,AD,AND,20,ISO 3166-2:AD,Europe,Southern Europe,NaN,150.0,39.0,NaN
6,Angola,AO,AGO,24,ISO 3166-2:AO,Africa,Sub-Saharan Africa,Middle Africa,2.0,202.0,17.0
7,Anguilla,AI,AIA,660,ISO 3166-2:AI,Americas,Latin America and the Caribbean,Caribbean,19.0,419.0,29.0
8,Antarctica,AQ,ATA,10,ISO 3166-2:AQ,NaN,NaN,NaN,NaN,NaN,NaN
9,Antigua and Barbuda,AG,ATG,28,ISO 3166-2:AG,Americas,Latin America and the Caribbean,Caribbean,19.0,419.0,29.0


In [9]:
print("---ISO code list---")
print("The number of countries:", ISO_code.shape[0])
print("The number of regions:", len(ISO_code['region'].value_counts().index))
print("The number of sub-regions:", len(ISO_code['sub-region'].value_counts().index))
print("The number of NaN in the region column:", np.sum(ISO_code['region'].isnull()))
print("The number of NaN in the sub-region column:", np.sum(ISO_code['sub-region'].isnull()))

---ISO code list---
The number of countries: 249
The number of regions: 5
The number of sub-regions: 17
The number of NaN in the region column: 1
The number of NaN in the sub-region column: 1


In [10]:
# save data as a database using SQLite

common_path = os.getcwd()
datpath = common_path + "/output/lausanne_marathon.db"

conn = sqlite3.connect(datpath) # establish a conenction to the SQLite database from python
cur = conn.cursor() # get a cursor object from the connection object

# Create a table named ISO_code with data
ISO_code.to_sql('ISO_code', conn, if_exists='replace')
    
IOC_code.to_sql('IOC_code', conn, if_exists='replace')
    
# Save (commit) the changes
conn.commit()

# close
conn.close()

# Web scraping

There are 3 categories: 42 km, 21 km, and 10 km.

## 2018 male 42km

In [11]:
url='https://services.datasport.com/2018/lauf/lamara/rang091.htm' # 2018 male 42 km
r = requests.get(url)

As you can see in the output of the next cell, the data is not tabulated. Numbers and letters are separated by spaces. Therefore, it is necessary to tabulate the data in a nice way.

In [12]:
idx_top=r.content.decode(r.encoding).find('--------------------------------------------------------------------------\r\n</font><font size="2">')
idx_bot=r.content.decode(r.encoding).find('\r\n\r\n\t\t\t\t\t<p><font size="2">Die Ergebnisse')

cont=r.content.decode(r.encoding)[idx_top+100:idx_bot].split('¦\r\n ')

print("Number of runners in the list:", len(cont)) 

cont[:3] # Top 3

Number of runners in the list: 1116


['  1. Hrioued Alaa              MAR 1985 F-St etienne                                               2:22.16,5       -----      (4) <a href="https://www.datasport.com/jump/20110/4/tc/dipl.htm" target="_blank">diplôme</a> <a href="https://www.datasport.com/jump/20110/4/tr/foto.htm" target="_blank">foto</a> <a href="https://www.datasport.com/jump/20110/4/tb/vide.htm" target="_blank">video</a>  42-H30   1.       3.22 ¦ 1:12.18     3.¦1:09.58     1.',
 '   2. Kumbi Bechere Tura        ETH 1979 F-Les Ulis                                                 2:23.50,8      1.34,3     (26) <a href="https://www.datasport.com/jump/20110/26/tv/dipl.htm" target="_blank">diplôme</a> <a href="https://www.datasport.com/jump/20110/26/ti/foto.htm" target="_blank">foto</a> <a href="https://www.datasport.com/jump/20110/26/tc/vide.htm" target="_blank">video</a>  42-H30   2.       3.24 ¦ 1:12.18     4.¦1:11.32     2.',
 '   3. Feleke Haillee Masha      ETH 1983 F-Aubiere                                        

Let us make a pandas dataframe.

In [13]:
df=pd.DataFrame(columns=['country', 'birth year', 'time', 'time_0-21km', 'time_21-42km'],index=range(len(cont))) # create a dataframe

for i in range(len(cont)):

    if cont[i].count('----') < 3:  # finishers

        # Time
        idx_tim1=cont[i].rfind('¦') 
        tim_latter=cont[i][idx_tim1+1:].strip().split()[0] # time 1/2 - 1

        hour=int(tim_latter.split(':')[0]) # hours
        minute=int(tim_latter.split(':')[1].split('.')[0]) # minute
        second=int(tim_latter.split(':')[1].split('.')[1]) # second

        tim_latter=hour * 60 + minute + second/60 # time 1/2 - 1 (minute)

        idx_tim2=cont[i][:idx_tim1].rfind('¦')
        tim_first=cont[i][idx_tim2+1:idx_tim1].split()[0] # time 0-1/2

        hour=int(tim_first.split(':')[0]) # hours
        minute=int(tim_first.split(':')[1].split('.')[0]) # minute
        second=int(tim_first.split(':')[1].split('.')[1]) # second

        tim_first=hour * 60 + minute + second/60 # time 0 - 1/2 (minute)

        df.iloc[i,3]=tim_first
        df.iloc[i,4]=tim_latter
        df.iloc[i,2]=tim_first+tim_latter

        # Country and birth year
        idx_cut = cont[i].find('<a href')

        j=0
        for d in cont[i][:idx_cut].split():
            if len(d) == 3 and d.isupper() == True:
                df.iloc[i,0]=d  # Country
                df.iloc[i,1]=int(cont[i][:idx_cut].split()[j+1]) # Birth year
                break
            j=j+1
        

    else: # non-finisher
        # Country and birth year
        idx_cut = cont[i].find('<a href')
        
        j=0
        for d in cont[i][:idx_cut].split():
            if len(d) == 3 and d.isupper() == True:
                df.iloc[i,0]=d  # Country
                df.iloc[i,1]=int(cont[i][:idx_cut].split()[j+1]) # Birth year
                break
            j=j+1



In [14]:
df_2018male42=df.copy()
df_2018male42.tail(30)

,country,birth year,time,time_0-21km,time_21-42km
1086,SUI,1958,375.733,156.45,219.283
1087,SUI,1962,377.95,165.05,212.9
1088,GER,1976,378.667,161.1,217.567
1089,SUI,1992,404.167,160.583,243.583
1090,FRA,1979,404.95,132.1,272.85
1091,SUI,1946,424.45,171.85,252.6
1092,FRA,1989,NaN,NaN,NaN
1093,AUS,1973,NaN,NaN,NaN
1094,SUI,2000,NaN,NaN,NaN
1095,SUI,1991,NaN,NaN,NaN


Let us replace the IOC country code by ISO country code.

In [15]:
list_unknown_country_male42=[]

for j in df_2018male42['country'].value_counts().index:
    if (j in ISO_code['alpha-3'].values)==False:
        if (j in IOC_code['NOC'].values.tolist()) == True:
            idx=IOC_code['NOC'].values.tolist().index(j)
            country_name = IOC_code['Country'][idx]
            idx=ISO_code['name'].values.tolist().index(country_name)
            iso_code=ISO_code['alpha-3'][idx]
            df_2018male42['country']=df_2018male42['country'].replace(j, iso_code)
        else:
            list_unknown_country_male42.append(j)

print("The following countries cannot be found in the list of the country code:")
print(list_unknown_country_male42)

The following countries cannot be found in the list of the country code:
['RSK']


The column "region" is added.

In [16]:
df_2018male42['region']=df_2018male42['country'].copy()

for j in range(df_2018male42['country'].shape[0]):
    if (df_2018male42['country'].iloc[j] in list_unknown_country_male42) == False:
        idx=ISO_code['alpha-3'].values.tolist().index(df_2018male42['country'].iloc[j])
        df_2018male42['region'].iloc[j]= ISO_code['region'].iloc[idx]
    else:
        df_2018male42['region'].iloc[j]=None 
        
df_2018male42=df_2018male42.iloc[:,[0,5,1,2,3,4]].copy() # change the order of columns
df_2018male42.rename(columns={'birth year':'age'}, inplace=True)
df_2018male42['age']=2018-df_2018male42['age'] # calculate age

df_2018male42.head()

,country,region,age,time,time_0-21km,time_21-42km
0,MAR,Africa,33,142.267,72.3,69.9667
1,ETH,Africa,39,143.833,72.3,71.5333
2,ETH,Africa,35,145,71.9667,73.0333
3,FRA,Europe,28,148.717,71.9667,76.75
4,POL,Europe,39,148.95,73.15,75.8


The next cell is used to save the dataframe as a csv file and database.

In [17]:
# save data as a csv file
#df_2018male42.to_csv(output_path + "/2018male42.csv")

# save data as a database using SQLite

common_path = os.getcwd()
datpath = common_path + "/output/lausanne_marathon.db"

conn = sqlite3.connect(datpath) # establish a conenction to the SQLite database from python
cur = conn.cursor() # get a cursor object from the connection object

# Create a table named male42km with data
df_2018male42.to_sql('male42km', conn, if_exists='replace')
     
# Save (commit) the changes
conn.commit()

# close
conn.close()

## 2018 female 42 km

In [18]:
url='https://services.datasport.com/2018/lauf/lamara/rang092.htm' # 2018 female 42 km
r = requests.get(url)

idx_top=r.content.decode(r.encoding).find('--------------------------------------------------------------------------\r\n</font><font size="2">')
idx_bot=r.content.decode(r.encoding).find('\r\n\r\n\t\t\t\t\t<p><font size="2">Die Ergebnisse')
cont=r.content.decode(r.encoding)[idx_top+100:idx_bot].split('¦\r\n ')
print("Number of runners in the list:", len(cont)) 

Number of runners in the list: 272


In [19]:
df=pd.DataFrame(columns=['country', 'birth year', 'time', 'time_0-21km', 'time_21-42km'],index=range(len(cont))) # create a dataframe

for i in range(len(cont)):

    if cont[i].count('----') < 3:  # finishers

        # Time
        idx_tim1=cont[i].rfind('¦') 
        tim_latter=cont[i][idx_tim1+1:].strip().split()[0] # time 1/2 - 1

        hour=int(tim_latter.split(':')[0]) # hours
        minute=int(tim_latter.split(':')[1].split('.')[0]) # minute
        second=int(tim_latter.split(':')[1].split('.')[1]) # second

        tim_latter=hour * 60 + minute + second/60 # time 1/2 - 1 (minute)

        idx_tim2=cont[i][:idx_tim1].rfind('¦')
        tim_first=cont[i][idx_tim2+1:idx_tim1].split()[0] # time 0-1/2

        hour=int(tim_first.split(':')[0]) # hours
        minute=int(tim_first.split(':')[1].split('.')[0]) # minute
        second=int(tim_first.split(':')[1].split('.')[1]) # second

        tim_first=hour * 60 + minute + second/60 # time 0 - 1/2 (minute)

        df.iloc[i,3]=tim_first
        df.iloc[i,4]=tim_latter
        df.iloc[i,2]=tim_first+tim_latter

        # Country and birth year
        idx_cut = cont[i].find('<a href')

        j=0
        for d in cont[i][:idx_cut].split():
            if len(d) == 3 and d.isupper() == True:
                df.iloc[i,0]=d  # Country
                df.iloc[i,1]=int(cont[i][:idx_cut].split()[j+1]) # Birth year
                break
            j=j+1
        

    else: # non-finisher
        # Country and birth year
        idx_cut = cont[i].find('<a href')
        
        j=0
        for d in cont[i][:idx_cut].split():
            if len(d) == 3 and d.isupper() == True:
                df.iloc[i,0]=d  # Country
                df.iloc[i,1]=int(cont[i][:idx_cut].split()[j+1]) # Birth year
                break
            j=j+1




In [20]:
df_2018female42=df.copy()
df_2018female42.tail(20)

,country,birth year,time,time_0-21km,time_21-42km
252,SUI,1966,329.9,134.333,195.567
253,AUT,1950,332.617,155.85,176.767
254,JPN,1948,337.533,153.017,184.517
255,JPN,1951,340.75,156.517,184.233
256,GER,1960,340.883,160.967,179.917
257,FRA,1997,342.067,144.8,197.267
258,FRA,1971,345.083,152.85,192.233
259,SUI,1998,347.1,150.217,196.883
260,FRA,1955,350.2,159.183,191.017
261,ITA,1963,352.517,165.933,186.583


In [21]:
list_unknown_country_female42=[]

for j in df_2018female42['country'].value_counts().index:
    if (j in ISO_code['alpha-3'].values)==False:
        if (j in IOC_code['NOC'].values.tolist()) == True:
            idx=IOC_code['NOC'].values.tolist().index(j)
            country_name = IOC_code['Country'][idx]
            idx=ISO_code['name'].values.tolist().index(country_name)
            iso_code=ISO_code['alpha-3'][idx]
            df_2018female42['country']=df_2018female42['country'].replace(j, iso_code)
        else:
            list_unknown_country_female42.append(j)

print("The following countries cannot be found in the list of the country code:")
print(list_unknown_country_female42)

The following countries cannot be found in the list of the country code:
[]


In [22]:
df_2018female42['region']=df_2018female42['country'].copy()

for j in range(df_2018female42['country'].shape[0]):
    if (df_2018female42['country'].iloc[j] in list_unknown_country_female42) == False:
        idx=ISO_code['alpha-3'].values.tolist().index(df_2018female42['country'].iloc[j])
        df_2018female42['region'].iloc[j]= ISO_code['region'].iloc[idx]
    else:
        df_2018female42['region'].iloc[j]=None 

df_2018female42=df_2018female42.iloc[:,[0,5,1,2,3,4]].copy() # change the order of columns
df_2018female42.rename(columns={'birth year':'age'}, inplace=True)
df_2018female42['age']=2018-df_2018female42['age'] # calculate age

df_2018female42.head()

,country,region,age,time,time_0-21km,time_21-42km
0,FRA,Europe,45,163.233,79.9833,83.25
1,CHE,Europe,22,164.217,81.2833,82.9333
2,UKR,Europe,32,180.617,85.3333,95.2833
3,DEU,Europe,28,184.833,91.1667,93.6667
4,FRA,Europe,42,187.8,90.5167,97.2833


In [24]:
# save data as a csv file
#df_2018female42.to_csv("2018female42.csv")

# save data as a database using SQLite
common_path = os.getcwd()
datpath = common_path + "/output/lausanne_marathon.db"

conn = sqlite3.connect(datpath) # establish a conenction to the SQLite database from python
cur = conn.cursor() # get a cursor object from the connection object

# Create a table named female42km with data
df_2018female42.to_sql('female42km', conn, if_exists='replace')
     
# Save (commit) the changes
conn.commit()

# close
conn.close()

## 2018 male 21 km

In [25]:
url='https://services.datasport.com/2018/lauf/lamara/rang094.htm' # 2018 male 21 km
r = requests.get(url)

In [26]:
idx_top=r.content.decode(r.encoding).find('--------------------------------------------------------------------------\r\n</font><font size="2">')
idx_bot=r.content.decode(r.encoding).find('\r\n\r\n\t\t\t\t\t<p><font size="2">Die Ergebnisse')

cont=r.content.decode(r.encoding)[idx_top+100:idx_bot].split('¦ \r\n ')

print("Number of runners in the list:", len(cont)) 

cont[:3] # Top 3

Number of runners in the list: 2858


['  1. Wodajo Alemayehu          ETH  1987 Lausanne                                                      1:06.56,8       -----     (32) <a href="https://www.datasport.com/jump/20110/32/ts/dipl.htm" target="_blank">diplôme</a> <a href="https://www.datasport.com/jump/20110/32/tf/foto.htm" target="_blank">foto</a> <a href="https://www.datasport.com/jump/20110/32/ty/vide.htm" target="_blank">video</a>  21-H30   1.       3.10 ',
 '   2. Tefera Mekonen            ETH  1992 Herrenschwanden              TV Länggasse                     1:07.08,4      0.11,6     (40) <a href="https://www.datasport.com/jump/20110/40/tr/dipl.htm" target="_blank">diplôme</a> <a href="https://www.datasport.com/jump/20110/40/te/foto.htm" target="_blank">foto</a> <a href="https://www.datasport.com/jump/20110/40/tx/vide.htm" target="_blank">video</a>  21-H20   1.       3.10 ',
 '   3. Leboeuf François          SUI  1985 Aigle                        BCVS Mount Asics Team            1:08.51,4      1.54,6     (33) <a hre

In [27]:
df=pd.DataFrame(columns=['country', 'birth year', 'time'],index=range(len(cont))) # create a dataframe

for i in range(len(cont)): 

    # country and birth year
    j=0
    idx=cont[i].rfind('(')

    for d in cont[i][:idx].strip().split():
        if len(d) == 3 and d.isupper() == True:
            idx_cut = cont[i].find('<a href')


            df.iloc[i,0]=d  # Country
            df.iloc[i,1]=int(cont[i][:idx_cut].split()[j+1]) # Birth year
            break
        j=j+1

    # time
    if ('km10' in cont[i])==False: #finishers
        idx=cont[i].rfind('(')
        idx_tim1 = cont[i][:idx].rfind('.')
        idx_tim2 = cont[i][:idx].rfind(':')

        if np.abs(idx_tim1- idx_tim2) == 3: # 1st runner and runners with the delay from the 1st runner is more than 1 hour
            if ('-----' in cont[i]) == True: # 1st runner
                idx_tim=cont[0].find('-----')
                time=cont[0][:idx_tim].split()      
                time=time[len(time)-1]

            else: # other runners
                time=cont[i][:idx_tim2-1].split()
                time=time[len(time)-1]

            hour=int(time[:time.find(':')])
            minute=int(time[time.find(':')+1:time.find('.')])
            second=int(time[time.find('.')+1:time.find(',')])

            time=hour * 60 + minute + second/60

        else: # the delay from the 1st runner is less than 1 hour
            time=cont[i][:idx_tim1-2].split()
            time=time[len(time)-1]

            hour=int(time[:time.find(':')])
            minute=int(time[time.find(':')+1:time.find('.')])
            second=int(time[time.find('.')+1:time.find(',')])

            time=hour * 60 + minute + second/60
        df.iloc[i,2]=time
        
    else:
        df.iloc[i,2]=None


In [28]:
df_2018male21=df.copy()
df_2018male21.tail(20)

,country,birth year,time
2838,RSK,1987,169.567
2839,RUS,1972,170.967
2840,SUI,1991,176.383
2841,SUI,1953,177.75
2842,SUI,1990,177.85
2843,SUI,1974,185.833
2844,SUI,1978,189.567
2845,AUS,1962,194.683
2846,ROM,1987,195.167
2847,SUI,1947,196.3


In [29]:
list_unknown_country_male21=[]

for j in df_2018male21['country'].value_counts().index:
    if (j in ISO_code['alpha-3'].values)==False:
        if (j in IOC_code['NOC'].values.tolist()) == True:
            idx=IOC_code['NOC'].values.tolist().index(j)
            country_name = IOC_code['Country'][idx]
            idx=ISO_code['name'].values.tolist().index(country_name)
            iso_code=ISO_code['alpha-3'][idx]
            df_2018male21['country']=df_2018male21['country'].replace(j, iso_code)
        else:
            list_unknown_country_male21.append(j)

print("The following countries cannot be found in the list of the country code:")
print(list_unknown_country_male21)

The following countries cannot be found in the list of the country code:
['RSK', 'EQU']


In [30]:
df_2018male21['region']=df_2018male21['country'].copy()

for j in range(df_2018male21['country'].shape[0]):
    if (df_2018male21['country'].iloc[j] in list_unknown_country_male21) == False:
        idx=ISO_code['alpha-3'].values.tolist().index(df_2018male21['country'].iloc[j])
        df_2018male21['region'].iloc[j]= ISO_code['region'].iloc[idx]
    else:
        df_2018male21['region'].iloc[j]=None 

df_2018male21=df_2018male21.iloc[:,[0,3,1,2]].copy() # change the order of columns
df_2018male21.rename(columns={'birth year':'age'}, inplace=True)
df_2018male21['age']=2018-df_2018male21['age'] # calculate age
        
df_2018male21.head()

,country,region,age,time
0,ETH,Africa,31,66.9333
1,ETH,Africa,26,67.1333
2,CHE,Europe,33,68.85
3,KEN,Africa,31,69.2333
4,MAR,Africa,40,69.9667


In [32]:
# save data as a csv file
#df_2018male21.to_csv("2018male21.csv")


# save data as a database using SQLite
common_path = os.getcwd()
datpath = common_path + "/output/lausanne_marathon.db"

conn = sqlite3.connect(datpath) # establish a conenction to the SQLite database from python
cur = conn.cursor() # get a cursor object from the connection object

# Create a table named male21km with data
df_2018male21.to_sql('male21km', conn, if_exists='replace')
     
# Save (commit) the changes
conn.commit()

# close
conn.close()

## 2018 female 21 km

In [33]:
url='https://services.datasport.com/2018/lauf/lamara/rang095.htm' # 2018 female 21 km
r = requests.get(url)

In [34]:
idx_top=r.content.decode(r.encoding).find('--------------------------------------------------------------------------\r\n</font><font size="2">')
idx_bot=r.content.decode(r.encoding).find('\r\n\r\n\t\t\t\t\t<p><font size="2">Die Ergebnisse')

cont=r.content.decode(r.encoding)[idx_top+100:idx_bot].split('¦ \r\n ')

print("Number of runners in the list:", len(cont)) 

cont[:3] # Top 3

Number of runners in the list: 1470


['  1. Geletu Israel Silass      ETH  1999 ETH-Ethiopie                                                  1:15.43,1       -----     (53) <a href="https://www.datasport.com/jump/20110/53/tv/dipl.htm" target="_blank">diplôme</a> <a href="https://www.datasport.com/jump/20110/53/ti/foto.htm" target="_blank">foto</a> <a href="https://www.datasport.com/jump/20110/53/tc/vide.htm" target="_blank">video</a>  21-JunF  1.       3.35 ',
 '   2. Hrebec Laura              SUI  1977 Muraz (Collombey)            CS 13 étoiles                    1:15.52,8      0.09,7     (51) <a href="https://www.datasport.com/jump/20110/51/tt/dipl.htm" target="_blank">diplôme</a> <a href="https://www.datasport.com/jump/20110/51/tg/foto.htm" target="_blank">foto</a> <a href="https://www.datasport.com/jump/20110/51/ta/vide.htm" target="_blank">video</a>  21-D40   1.       3.35 ',
 '   3. Frei Melina               SUI  1989 Dübendorf                                                     1:20.06,2      4.23,1     (52) <a hre

In [35]:
df=pd.DataFrame(columns=['country', 'birth year', 'time'],index=range(len(cont))) # create a dataframe

for i in range(len(cont)): 

    # country and birth year
    j=0
    idx=cont[i].rfind('(')

    for d in cont[i][:idx].strip().split():
        if len(d) == 3 and d.isupper() == True:
            idx_cut = cont[i].find('<a href')


            df.iloc[i,0]=d  # Country
            df.iloc[i,1]=int(cont[i][:idx_cut].split()[j+1]) # Birth year
            break
        j=j+1

    # time
    if ('km10' in cont[i])==False: #finishers
        idx=cont[i].rfind('(')
        idx_tim1 = cont[i][:idx].rfind('.')
        idx_tim2 = cont[i][:idx].rfind(':')

        if np.abs(idx_tim1- idx_tim2) == 3: # 1st runner and runners with the delay from the 1st runner is more than 1 hour
            if ('-----' in cont[i]) == True: # 1st runner
                idx_tim=cont[0].find('-----')
                time=cont[0][:idx_tim].split()      
                time=time[len(time)-1]

            else: # other runners
                time=cont[i][:idx_tim2-1].split()
                time=time[len(time)-1]

            hour=int(time[:time.find(':')])
            minute=int(time[time.find(':')+1:time.find('.')])
            second=int(time[time.find('.')+1:time.find(',')])

            time=hour * 60 + minute + second/60

        else: # the delay from the 1st runner is less than 1 hour
            time=cont[i][:idx_tim1-2].split()
            time=time[len(time)-1]

            hour=int(time[:time.find(':')])
            minute=int(time[time.find(':')+1:time.find('.')])
            second=int(time[time.find('.')+1:time.find(',')])

            time=hour * 60 + minute + second/60
        df.iloc[i,2]=time
        
    else:
        df.iloc[i,2]=None



In [36]:
df_2018female21=df.copy()
df_2018female21.tail(20)

,country,birth year,time
1450,SUI,1978,169.833
1451,CHN,1999,171.033
1452,SUI,1996,171.033
1453,BEL,1959,172.683
1454,ITA,1969,173.283
1455,GBR,1967,173.983
1456,ITA,1972,174.283
1457,GBR,1990,175.667
1458,SUI,1995,176.25
1459,FRA,1991,177.95


In [37]:
list_unknown_country_female21=[]

for j in df_2018female21['country'].value_counts().index:
    if (j in ISO_code['alpha-3'].values)==False:
        if (j in IOC_code['NOC'].values.tolist()) == True:
            idx=IOC_code['NOC'].values.tolist().index(j)
            country_name = IOC_code['Country'][idx]
            idx=ISO_code['name'].values.tolist().index(country_name)
            iso_code=ISO_code['alpha-3'][idx]
            df_2018female21['country']=df_2018female21['country'].replace(j, iso_code)
        else:
            list_unknown_country_female21.append(j)

print("The following countries cannot be found in the list of the country code:")
print(list_unknown_country_female21)

The following countries cannot be found in the list of the country code:
[]


In [38]:
df_2018female21['region']=df_2018female21['country'].copy()

for j in range(df_2018female21['country'].shape[0]):
    if (df_2018female21['country'].iloc[j] in list_unknown_country_female21) == False:
        idx=ISO_code['alpha-3'].values.tolist().index(df_2018female21['country'].iloc[j])
        df_2018female21['region'].iloc[j]= ISO_code['region'].iloc[idx]
    else:
        df_2018female21['region'].iloc[j]=None 

df_2018female21=df_2018female21.iloc[:,[0,3,1,2]].copy() # change the order of columns
df_2018female21.rename(columns={'birth year':'age'}, inplace=True)
df_2018female21['age']=2018-df_2018female21['age'] # calculate age

df_2018female21.head()

,country,region,age,time
0,ETH,Africa,19,75.7167
1,CHE,Europe,41,75.8667
2,CHE,Europe,29,80.1
3,CHE,Europe,33,82.45
4,CHE,Europe,41,82.9


In [39]:
# save data as a csv file
#df_2018female21.to_csv("2018female21.csv")

# save data as a database using SQLite

common_path = os.getcwd()
datpath = common_path + "/output/lausanne_marathon.db"

conn = sqlite3.connect(datpath) # establish a conenction to the SQLite database from python
cur = conn.cursor() # get a cursor object from the connection object

# Create a table named female21km with data
df_2018female21.to_sql('female21km', conn, if_exists='replace')
     
# Save (commit) the changes
conn.commit()

# close
conn.close()

## 2018 male 10 km

In [40]:
url='https://services.datasport.com/2018/lauf/lamara/rang096.htm' # 2018 male 10 km
r = requests.get(url)

In [41]:
idx_top=r.content.decode(r.encoding).find('--------------------------------------------------------------------------\r\n</font><font size="2">')
idx_bot=r.content.decode(r.encoding).find('\r\n\r\n\t\t\t\t\t<p><font size="2">Die Ergebnisse')

cont=r.content.decode(r.encoding)[idx_top+100:idx_bot].split('¦ \r\n ')

print("Number of runners in the list:", len(cont)) 

cont[:3] # Top 3

Number of runners in the list: 2404


['  1. Matheka Bernard           KEN  1988 Malleray                                                        29.09,5       -----     (61) <a href="https://www.datasport.com/jump/20110/61/tu/dipl.htm" target="_blank">diplôme</a> <a href="https://www.datasport.com/jump/20110/61/th/foto.htm" target="_blank">foto</a> <a href="https://www.datasport.com/jump/20110/61/tb/vide.htm" target="_blank">video</a>  10-H30   1.       2.54 ',
 '   2. Meftah El Khair Mohammed  SUI  1989 Genève                                                          29.17,8      0.08,3     (74) <a href="https://www.datasport.com/jump/20110/74/ty/dipl.htm" target="_blank">diplôme</a> <a href="https://www.datasport.com/jump/20110/74/tl/foto.htm" target="_blank">foto</a> <a href="https://www.datasport.com/jump/20110/74/tf/vide.htm" target="_blank">video</a>  10-H20   1.       2.55 ',
 '   3. El Jaddar Ahmed           MAR  1986 Basel                        TV Riehen                          29.49,0      0.39,5     (62) <a hre

In [42]:
df=pd.DataFrame(columns=['country', 'birth year', 'time'],index=range(len(cont))) # create a dataframe

for i in range(len(cont)): 
    # country and birth year
    j=0
    idx=cont[i].rfind('(')

    for d in cont[i][:idx].strip().split():
        if len(d) == 3 and d.isupper() == True:
            df.iloc[i,0]=d  # Country
            if cont[i][:idx].strip().split()[j+1] != '????':
                df.iloc[i,1]=int(cont[i][:idx].strip().split()[j+1]) # Birth year
            else:
                df.iloc[i,1]=None
            break
        j=j+1

    # time
    time=cont[i][:idx].strip().split()
    time=time[len(time)-2]

    if time.find(':') == -1: # faster than 1 hour
        hour=0
        minute=int(time[:time.find('.')])
        second=int(time[time.find('.')+1:time.find(',')])
    else: # slower than 1 hour
        hour=int(time[:time.find(':')])
        minute=int(time[time.find(':')+1:time.find('.')])
        second=int(time[time.find('.')+1:time.find(',')])

    time=hour*60 + minute+second/60
    df.iloc[i,2]= time

In [43]:
df_2018male10=df.copy()
df_2018male10.tail(10)

,country,birth year,time
2394,USA,1968,90.7667
2395,SUI,1951,90.8
2396,SUI,1968,92.2667
2397,SUI,1976,95.8667
2398,SUI,1957,99.05
2399,CHI,1999,99.2833
2400,CHI,1999,99.3667
2401,SUI,1941,100.45
2402,HKG,1999,102.067
2403,HKG,1998,105.467


In [44]:
list_unknown_country_male10=[]

for j in df_2018male10['country'].value_counts().index:
    if (j in ISO_code['alpha-3'].values)==False:
        if (j in IOC_code['NOC'].values.tolist()) == True:
            idx=IOC_code['NOC'].values.tolist().index(j)
            country_name = IOC_code['Country'][idx]
            idx=ISO_code['name'].values.tolist().index(country_name)
            iso_code=ISO_code['alpha-3'][idx]
            df_2018male10['country']=df_2018male10['country'].replace(j, iso_code)
        else:
            list_unknown_country_male10.append(j)

print("The following countries cannot be found in the list of the country code:")
print(list_unknown_country_male10)

The following countries cannot be found in the list of the country code:
['JNP', 'RSK']


In [45]:
df_2018male10['region']=df_2018male10['country'].copy()

for j in range(df_2018male10['country'].shape[0]):
    if (df_2018male10['country'].iloc[j] in list_unknown_country_male10) == False:
        idx=ISO_code['alpha-3'].values.tolist().index(df_2018male10['country'].iloc[j])
        df_2018male10['region'].iloc[j]= ISO_code['region'].iloc[idx]
    else:
        df_2018male10['region'].iloc[j]=None 

df_2018male10=df_2018male10.iloc[:,[0,3,1,2]].copy() # change the order of columns
df_2018male10.rename(columns={'birth year':'age'}, inplace=True)
df_2018male10['age']=2018-df_2018male10['age'] # calculate age

df_2018male10.head()

,country,region,age,time
0,KEN,Africa,30,29.15
1,CHE,Europe,29,29.2833
2,MAR,Africa,32,29.8167
3,FRA,Europe,34,30.0167
4,KEN,Africa,25,30.0333


In [46]:
# save data as a csv file
#df_2018male10.to_csv("2018male10.csv")

# save data as a database using SQLite

common_path = os.getcwd()
datpath = common_path + "/output/lausanne_marathon.db"

conn = sqlite3.connect(datpath) # establish a conenction to the SQLite database from python
cur = conn.cursor() # get a cursor object from the connection object

# Create a table named male10km with data
df_2018male10.to_sql('male10km', conn, if_exists='replace')
     
# Save (commit) the changes
conn.commit()

# close
conn.close()

## 2018 female 10 km

In [47]:
url='https://services.datasport.com/2018/lauf/lamara/rang097.htm' # 2018 male 10 km
r = requests.get(url)

In [48]:
idx_top=r.content.decode(r.encoding).find('--------------------------------------------------------------------------\r\n</font><font size="2">')
idx_bot=r.content.decode(r.encoding).find('\r\n\r\n\t\t\t\t\t<p><font size="2">Die Ergebnisse')

cont=r.content.decode(r.encoding)[idx_top+100:idx_bot].split('¦ \r\n ')

print("Number of runners in the list:", len(cont)) 

cont[:3] # Top 3

Number of runners in the list: 2370


['  1. Jeronoh Mercyline         KEN  1992 F-Annecy                                                        33.37,3       -----     (93) <a href="https://www.datasport.com/jump/20110/93/ta/dipl.htm" target="_blank">diplôme</a> <a href="https://www.datasport.com/jump/20110/93/tm/foto.htm" target="_blank">foto</a> <a href="https://www.datasport.com/jump/20110/93/tg/vide.htm" target="_blank">video</a>  10-D20   1.       3.21 ',
 '   2. Egger Nicole              SUI  1985 Langenthal                   LV Langenthal                      34.04,1      0.26,8  (10085) <a href="https://www.datasport.com/jump/20110/10085/qh/dipl.htm" target="_blank">diplôme</a> <a href="https://www.datasport.com/jump/20110/10085/qw/foto.htm" target="_blank">foto</a> <a href="https://www.datasport.com/jump/20110/10085/qg/vide.htm" target="_blank">video</a>  10-D30   1.       3.24 ',
 '   3. Lemay Virginie            FRA  1982 F-Cran Gevrier               Sport Attitude Scuderi peintur     36.19,5      2.42,2     (9

In [49]:
df=pd.DataFrame(columns=['country', 'birth year', 'time'],index=range(len(cont))) # create a dataframe

for i in range(len(cont)): 
    # country and birth year
    j=0
    idx=cont[i].rfind('(')

    for d in cont[i][:idx].strip().split():
        if len(d) == 3 and d.isupper() == True:
            df.iloc[i,0]=d  # Country
            if cont[i][:idx].strip().split()[j+1] != '????':
                df.iloc[i,1]=int(cont[i][:idx].strip().split()[j+1]) # Birth year
            else:
                df.iloc[i,1]=None
            break
        j=j+1

    # time
    time=cont[i][:idx].strip().split()
    time=time[len(time)-2]

    if time.find(':') == -1: # faster than 1 hour
        hour=0
        minute=int(time[:time.find('.')])
        second=int(time[time.find('.')+1:time.find(',')])
    else: # slower than 1 hour
        hour=int(time[:time.find(':')])
        minute=int(time[time.find(':')+1:time.find('.')])
        second=int(time[time.find('.')+1:time.find(',')])

    time=hour*60 + minute+second/60
    df.iloc[i,2]= time

In [50]:
df_2018female10=df.copy()
df_2018female10.tail(10)

,country,birth year,time
2360,HKG,1999,102.067
2361,JPN,1966,104.85
2362,JPN,1952,104.917
2363,CHI,1998,105.417
2364,CHI,1997,105.45
2365,CHI,1999,105.5
2366,CHI,2000,105.567
2367,CHI,1999,105.583
2368,HKG,1998,106.35
2369,THA,1998,107.883


In [51]:
list_unknown_country_female10=[]

for j in df_2018female10['country'].value_counts().index:
    if (j in ISO_code['alpha-3'].values)==False:
        if (j in IOC_code['NOC'].values.tolist()) == True:
            idx=IOC_code['NOC'].values.tolist().index(j)
            country_name = IOC_code['Country'][idx]
            idx=ISO_code['name'].values.tolist().index(country_name)
            iso_code=ISO_code['alpha-3'][idx]
            df_2018female10['country']=df_2018female10['country'].replace(j, iso_code)
        else:
            list_unknown_country_female10.append(j)

print("The following countries cannot be found in the list of the country code:")
print(list_unknown_country_female10)

The following countries cannot be found in the list of the country code:
['JNP']


In [52]:
df_2018female10['region']=df_2018female10['country'].copy()

for j in range(df_2018female10['country'].shape[0]):
    if (df_2018female10['country'].iloc[j] in list_unknown_country_female10) == False:
        idx=ISO_code['alpha-3'].values.tolist().index(df_2018female10['country'].iloc[j])
        df_2018female10['region'].iloc[j]= ISO_code['region'].iloc[idx]
    else:
        df_2018female10['region'].iloc[j]=None 

df_2018female10=df_2018female10.iloc[:,[0,3,1,2]].copy() # change the order of columns
df_2018female10.rename(columns={'birth year':'age'}, inplace=True)
df_2018female10['age']=2018-df_2018female10['age'] # calculate age        

df_2018female10.head()

,country,region,age,time
0,KEN,Africa,26,33.6167
1,CHE,Europe,33,34.0667
2,FRA,Europe,36,36.3167
3,CHE,Europe,32,36.5
4,FRA,Europe,26,36.5167


In [53]:
# save data as a csv file
#df_2018female10.to_csv("2018female10.csv")

# save data as a database using SQLite

common_path = os.getcwd()
datpath = common_path + "/output/lausanne_marathon.db"

conn = sqlite3.connect(datpath) # establish a conenction to the SQLite database from python
cur = conn.cursor() # get a cursor object from the connection object

# Create a table named female10km with data
df_2018female10.to_sql('female10km', conn, if_exists='replace')
     
# Save (commit) the changes
conn.commit()

# close
conn.close()

# Combining data

## 42 km

The dataframes for male and female runners are combined.

In [54]:
df_2018all42=df_2018male42.iloc[:,:4].copy()
df_2018all42['gender']='male'

temp=df_2018female42.iloc[:,:4].copy()
temp['gender']='female'

df_2018all42=pd.concat([df_2018all42, temp]).reset_index(drop=True) # combine data for male and female runners.
df_2018all42['category']='42km' # add a category

df_2018all42.sort_values('time',inplace=True) # sort values based on the time
df_2018all42.reset_index(drop=True, inplace=True) # change index
df_2018all42.head(100)

,country,region,age,time,gender,category
0,MAR,Africa,33,142.267,male,42km
1,ETH,Africa,39,143.833,male,42km
2,ETH,Africa,35,145,male,42km
3,FRA,Europe,28,148.717,male,42km
4,POL,Europe,39,148.95,male,42km
5,ETH,Africa,31,155.367,male,42km
6,IND,Asia,28,155.467,male,42km
7,CHE,Europe,35,156.917,male,42km
8,FRA,Europe,38,157.417,male,42km
9,FRA,Europe,28,158.4,male,42km


## 21 km

The dataframes for male and female runners are combined.

In [55]:
df_2018all21=df_2018male21.copy()
df_2018all21['gender']='male'

temp=df_2018female21.copy()
temp['gender']='female'

df_2018all21=pd.concat([df_2018all21, temp]).reset_index(drop=True) # combine data for male and female runners.
df_2018all21['category']='21km' # add a category

df_2018all21.sort_values('time',inplace=True) # sort values based on the time
df_2018all21.reset_index(drop=True, inplace=True) # change index
df_2018all21.head(100)

,country,region,age,time,gender,category
0,ETH,Africa,31,66.9333,male,21km
1,ETH,Africa,26,67.1333,male,21km
2,CHE,Europe,33,68.85,male,21km
3,KEN,Africa,31,69.2333,male,21km
4,MAR,Africa,40,69.9667,male,21km
5,CHE,Europe,37,70.8333,male,21km
6,CHE,Europe,27,71.6667,male,21km
7,GBR,Europe,32,71.8833,male,21km
8,FRA,Europe,35,72.2167,male,21km
9,CHE,Europe,32,73.2167,male,21km


## 10 km

The dataframes for male and female runners are combined.

In [56]:
df_2018all10=df_2018male10.copy()
df_2018all10['gender']='male'

temp=df_2018female10.copy()
temp['gender']='female'

df_2018all10=pd.concat([df_2018all10, temp]).reset_index(drop=True) # combine data for male and female runners.
df_2018all10['category']='10km' # add a category

df_2018all10.sort_values('time',inplace=True) # sort values based on the time
df_2018all10.reset_index(drop=True, inplace=True) # change index
df_2018all10.head(200)

,country,region,age,time,gender,category
0,KEN,Africa,30,29.15,male,10km
1,CHE,Europe,29,29.2833,male,10km
2,MAR,Africa,32,29.8167,male,10km
3,FRA,Europe,34,30.0167,male,10km
4,KEN,Africa,25,30.0333,male,10km
5,ETH,Africa,25,30.05,male,10km
6,MAR,Africa,33,30.15,male,10km
7,CHE,Europe,29,30.1667,male,10km
8,CHE,Europe,30,30.4333,male,10km
9,ETH,Africa,32,30.55,male,10km


## All categories

All dataframes are combined.

In [57]:
df_2018all=pd.concat([df_2018all42,df_2018all21,df_2018all10]).reset_index(drop=True)
df_2018all

,country,region,age,time,gender,category
0,MAR,Africa,33,142.267,male,42km
1,ETH,Africa,39,143.833,male,42km
2,ETH,Africa,35,145,male,42km
3,FRA,Europe,28,148.717,male,42km
4,POL,Europe,39,148.95,male,42km
5,ETH,Africa,31,155.367,male,42km
6,IND,Asia,28,155.467,male,42km
7,CHE,Europe,35,156.917,male,42km
8,FRA,Europe,38,157.417,male,42km
9,FRA,Europe,28,158.4,male,42km


## Saving data

In [58]:
# save data as a csv file
df_2018all42.to_csv(output_path + "/lausanne2018_all42km.csv")
df_2018all21.to_csv(output_path + "/lausanne2018_all21km.csv")
df_2018all10.to_csv(output_path + "/lausanne2018_all10km.csv")
df_2018all.to_csv(output_path + "/lausanne2018_all_categories.csv")

# save data as a database using SQLite
common_path = os.getcwd()
datpath = common_path + "/output/lausanne_marathon.db"

conn = sqlite3.connect(datpath) # establish a conenction to the SQLite database from python
cur = conn.cursor() # get a cursor object from the connection object

# Create tables with data
df_2018all42.to_sql('all42km', conn, if_exists='replace')
df_2018all21.to_sql('all21km', conn, if_exists='replace')
df_2018all10.to_sql('all10km', conn, if_exists='replace')
df_2018all.to_sql('all_categories', conn, if_exists='replace')
     
# Save (commit) the changes
conn.commit()

# close
conn.close()

# Summarized data

In [60]:
# number of foreign runners
num_male_foreign_runner_10km=df_2018male10[df_2018male10['country'] != 'CHE'].shape[0] 
num_female_foreign_runner_10km=df_2018female10[df_2018female10['country'] != 'CHE'].shape[0]
num_total_foreign_runner_10km=num_male_foreign_runner_10km+num_female_foreign_runner_10km

num_male_foreign_runner_21km=df_2018male21[df_2018male21['country'] != 'CHE'].shape[0]
num_female_foreign_runner_21km=df_2018female21[df_2018female21['country'] != 'CHE'].shape[0]
num_total_foreign_runner_21km=num_male_foreign_runner_21km+num_female_foreign_runner_21km

num_male_foreign_runner_42km=df_2018male42[df_2018male42['country'] != 'CHE'].shape[0]
num_female_foreign_runner_42km=df_2018female42[df_2018female42['country'] != 'CHE'].shape[0]
num_total_foreign_runner_42km=num_male_foreign_runner_42km+num_female_foreign_runner_42km

num_male_foreign_runner_total=num_male_foreign_runner_10km + num_male_foreign_runner_21km + num_male_foreign_runner_42km
num_female_foreign_runner_total=num_female_foreign_runner_10km + num_female_foreign_runner_21km + num_female_foreign_runner_42km

# number of participants
total_participants=df_2018male42.shape[0]+df_2018female42.shape[0]+df_2018male21.shape[0]+df_2018female21.shape[0]+df_2018male10.shape[0]+df_2018female10.shape[0]
total_male_participants=df_2018male42.shape[0]+df_2018male21.shape[0]+df_2018male10.shape[0]
total_female_participants=df_2018female42.shape[0]+df_2018female21.shape[0]+df_2018female10.shape[0]

#42 km participants
total_participants_42km=df_2018male42.shape[0]+df_2018female42.shape[0]
total_male_participants_42km=df_2018male42.shape[0]
total_female_participants_42km=df_2018female42.shape[0]

#42 km nonfinisher
male_nonfinisher_42km=np.sum(df_2018male42['time'].isnull())
female_nonfinisher_42km=np.sum(df_2018female42['time'].isnull())
total_nonfinisher_42km=male_nonfinisher_42km+female_nonfinisher_42km

#21 km participants
total_participants_21km=df_2018male21.shape[0]+df_2018female21.shape[0]
total_male_participants_21km=df_2018male21.shape[0]
total_female_participants_21km=df_2018female21.shape[0]

# 21 km nonfinisher
male_nonfinisher_21km=np.sum(df_2018male21['time'].isnull())
female_nonfinisher_21km=np.sum(df_2018female21['time'].isnull())
total_nonfinisher_21km=male_nonfinisher_21km+female_nonfinisher_21km

#10 km participants
total_participants_10km=df_2018male10.shape[0]+df_2018female10.shape[0]
total_male_participants_10km=df_2018male10.shape[0]
total_female_participants_10km=df_2018female10.shape[0]

# Data frame
df_summary=pd.DataFrame(
{ 'Category':['total','42km','21km','10km'],
  'Total_number_of_participants':
 [total_participants_42km+total_participants_21km+total_participants_10km, total_participants_42km,total_participants_21km,total_participants_10km],
  'Number_of_male_participants': 
 [total_male_participants_42km+total_male_participants_21km+total_male_participants_10km,total_male_participants_42km,total_male_participants_21km,total_male_participants_10km],
  'Number_of_female_participants':
 [total_female_participants_42km+total_female_participants_21km+total_female_participants_10km,total_female_participants_42km,total_female_participants_21km,total_female_participants_10km],
  'Number_of_foreign_runner':
 [num_total_foreign_runner_42km+num_total_foreign_runner_21km+num_total_foreign_runner_10km,num_total_foreign_runner_42km,num_total_foreign_runner_21km,num_total_foreign_runner_10km],
  'Number_of_male_foreign_runner':
 [num_male_foreign_runner_42km+num_male_foreign_runner_21km+num_male_foreign_runner_10km,num_male_foreign_runner_42km,num_male_foreign_runner_21km,num_male_foreign_runner_10km],
  'Number_of_female_foreign_runner':
 [num_female_foreign_runner_42km+num_female_foreign_runner_21km+num_female_foreign_runner_10km,num_female_foreign_runner_42km,num_female_foreign_runner_21km,num_female_foreign_runner_10km],
 'Number_of_nonfinisher': 
 [total_nonfinisher_42km+total_nonfinisher_21km,total_nonfinisher_42km,total_nonfinisher_21km,0],
  'Number_of_male_nonfinisher':
 [male_nonfinisher_42km+ female_nonfinisher_21km,male_nonfinisher_42km, female_nonfinisher_21km,0],
  'Number_of_female_nonfinisher': 
 [female_nonfinisher_42km+ female_nonfinisher_21km,female_nonfinisher_42km, female_nonfinisher_21km,0],
})

df_summary=df_summary.set_index('Category')

# save the data as a csv file
df_summary.to_csv(output_path + "/lausanne2018_summary.csv")

# save data as a database using SQLite
common_path = os.getcwd()
datpath = common_path + "/output/lausanne_marathon.db"

conn = sqlite3.connect(datpath) # establish a conenction to the SQLite database from python
cur = conn.cursor() # get a cursor object from the connection object

# Create tables with data
df_summary.to_sql('summary', conn, if_exists='replace')
     
# Save (commit) the changes
conn.commit()

# close
conn.close()


df_summary

,Total_number_of_participants,Number_of_male_participants,Number_of_female_participants,Number_of_foreign_runner,Number_of_male_foreign_runner,Number_of_female_foreign_runner,Number_of_nonfinisher,Number_of_male_nonfinisher,Number_of_female_nonfinisher
Category,,,,,,,,,
total,10490,6378,4112,4403,2834,1569,45,27,11
42km,1388,1116,272,805,633,172,32,24,8
21km,4328,2858,1470,1961,1299,662,13,3,3
10km,4774,2404,2370,1637,902,735,0,0,0
